In [177]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from SoSell import logger
import time
import os


In [178]:
def initialize_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    service = Service('/Users/Stark0104/Desktop/Coding/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [179]:
def login_to_instagram(driver, url):
    driver.get(url)
    try:
        # Wait for the password field to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'password'))
        )
        logger.info("Password field detected.")

        # Wait for the 'Close' button (if it exists, such as for a pop-up)  # Optional: Allow time for any animation or DOM update

        # Use JavaScript to click for reliability
        

    except Exception as e:
        logger.info(f"Error during login: {e}")


In [180]:
def scrape_instagram_post(driver, url):
    try:
        driver.get(url)

        # Close initial pop-up (if any)
        try:
            close_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @tabindex="0"]'))
            )
            logger.info("Close button detected, attempting to click.")
            
            # Ensure the element is within the viewport before clicking
            # driver.execute_script("arguments[0].scrollIntoView(true);", close_button)
            driver.execute_script("arguments[0].click();", close_button)
            logger.info("Close button clicked.")
        except Exception as e:
            logger.warning(f"No close button found or clickable: {e}")

        # Dismiss secondary pop-up, if any
        try:
            close_secondary = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//span[@aria-label="Close"]'))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", close_secondary)
            driver.execute_script("arguments[0].click();", close_secondary)
            logger.info("Secondary close button clicked.")
        except Exception as e:
            logger.warning(f"No secondary close button found: {e}")

        logger.info("Waiting for post content to load...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@dir='auto' and contains(text(), 'Follow')]"))
        )
        logger.info("Post loaded, ready to scrape.")

        post_data = {}

        # Look for images in the post
        logger.info("Looking for image elements...")
        try:
            image_containers = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, '_aagv')]//img[@crossorigin='anonymous']"))
            )
            post_data['image_urls'] = [img.get_attribute('srcset') for img in image_containers]
            logger.info(f"Image elements found: {len(post_data['image_urls'])}")
        except Exception as e:
            logger.error(f"Error finding image elements: {e}")
            post_data['image_urls'] = []

        # Look for the caption
        logger.info("Looking for caption...")
        caption_elements = driver.find_elements(By.XPATH, '//div[@class="_a9zr"]')
        if caption_elements:
            post_data['caption'] = caption_elements[0].text
            logger.info("Caption found.")
        else:
            logger.warning("Caption element not found.")
            post_data['caption'] = "No caption found."

        logger.info("Scraped data successfully.")
        return post_data

    except Exception as e:
        logger.error(f"Error scraping post: {e}")
        logger.error(f"Post URL: {url}")
        return None

In [181]:
if __name__ == "__main__":
    post_url = "https://www.instagram.com/p/CrwY_Iuh6mM/"
    
    driver = initialize_driver()
    
    
    login_to_instagram(driver, post_url)
     
        
    data = scrape_instagram_post(driver, post_url)
    print(data)
    
    driver.quit()


[2024-10-22 17:26:14,511: INFO: 246606317: Password field detected.]
[2024-10-22 17:26:15,177: INFO: 2694907801: Close button detected, attempting to click.]
[2024-10-22 17:26:15,183: INFO: 2694907801: Close button clicked.]
[2024-10-22 17:26:16,235: INFO: 2694907801: Secondary close button clicked.]
[2024-10-22 17:26:16,236: INFO: 2694907801: Waiting for post content to load...]
[2024-10-22 17:26:16,245: INFO: 2694907801: Post loaded, ready to scrape.]
[2024-10-22 17:26:16,246: INFO: 2694907801: Looking for image elements...]
[2024-10-22 17:26:16,271: INFO: 2694907801: Image elements found: 2]
[2024-10-22 17:26:16,271: INFO: 2694907801: Looking for caption...]
[2024-10-22 17:26:16,298: INFO: 2694907801: Caption found.]
[2024-10-22 17:26:16,299: INFO: 2694907801: Scraped data successfully.]
{'image_urls': ['https://instagram.fbho3-2.fna.fbcdn.net/v/t51.29350-15/344255752_183809894595011_245967690214186054_n.webp?stp=dst-jpg_e35_s640x640_sh0.08&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi

In [182]:
data

{'image_urls': ['https://instagram.fbho3-2.fna.fbcdn.net/v/t51.29350-15/344255752_183809894595011_245967690214186054_n.webp?stp=dst-jpg_e35_s640x640_sh0.08&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDI0eDEwMjQuc2RyLmYyOTM1MC5kZWZhdWx0X2ltYWdlIn0&_nc_ht=instagram.fbho3-2.fna.fbcdn.net&_nc_cat=111&_nc_ohc=F_4dzH_cRj8Q7kNvgE4lje_&_nc_gid=e1977c1468494a27a43ad8a0c4f298b3&edm=ANTKIIoBAAAA&ccb=7-5&oh=00_AYDopOj0hgKTn-j1vN3RLmedBx-qyq1Tba0zI82UXtye9A&oe=671D6FEC&_nc_sid=d885a2 640w,https://instagram.fbho3-2.fna.fbcdn.net/v/t51.29350-15/344255752_183809894595011_245967690214186054_n.webp?stp=dst-jpg_e35_s750x750_sh0.08&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDI0eDEwMjQuc2RyLmYyOTM1MC5kZWZhdWx0X2ltYWdlIn0&_nc_ht=instagram.fbho3-2.fna.fbcdn.net&_nc_cat=111&_nc_ohc=F_4dzH_cRj8Q7kNvgE4lje_&_nc_gid=e1977c1468494a27a43ad8a0c4f298b3&edm=ANTKIIoBAAAA&ccb=7-5&oh=00_AYAuFdcDqh0m_DDO2tdZkWnmnJcbF1sw-CL9LEHkOdhDrA&oe=671D6FEC&_nc_sid=d885a2 750w,https://instagram.fbho3-2.fna.fbcdn.net/v/t51.29350-1